##### Quick Intro
The dataset is scraped from https://www.appliancerepair.net/dishwasher-repair-1.html and follow-on pages.

Update Oct 23, the page is no longer available, but the chunking is already done, so we're OK

Scraper code is at scraper.py.

The embeddings are generated using the [SentenceTransformer](https://www.sbert.net/) package (no finetuning)

The overall flow is from 
https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

The difference is, instead of using OpenAI Curie to create the embeddings, I create embeddings using SBERT and use cosine simialrity to retrieve relevant pieces of text.

You'll need an OpenAI key

In [1]:
%pip install pandas -q
%pip install openai -q
%pip install getpass4
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 265.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 340.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import openai
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer, util
print(f'openai api version: {openai.__version__}')

openai api version: 1.52.2


In [5]:
def get_embedding(text: str, model: SentenceTransformer) -> list[float]:
    embeddings = model.encode([text])
    return embeddings

def get_doc_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def get_query_embedding(text: str, model: SentenceTransformer) -> list[float]:
    return get_embedding(text, model)

def compute_doc_embeddings(df: pd.DataFrame, model: SentenceTransformer) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe.
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content.replace("\n", " "), model) for idx, r in df.iterrows()
    }

def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }


In [6]:
# The scraper generated the csv file below
df = pd.read_csv('dish-washer-data.csv')
df["tokens"] = pd.to_numeric(df["tokens"])  # convert column "tokens" of a DataFrame
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(10)
# TODO get some stats on max/min content length


149 rows in the data.


content  tokens
title     heading                                                           
Chapter_5 273       Feel the action of the flapper valve. If it i...      96
Chapter_2 64        It's important to know that washing dishes in...      80
Chapter_5 331       The pump, motor and upper and lower spray arm...     104
Chapter_4 190       There are two general areas where you will co...      77
Chapter_3 163       CLOGGING OF THE DISHWASHER WATER SYSTEM: The ...     123
Chapter_5 279       Your appliance parts dealer has an impeller a...      90
Chapter_6 377       Since a burnt out bimetal element is the most...     194
Chapter_5 284       There are two different impeller kits for two...      80
          301       When you get the pump/motor unit back into pl...      80
Chapter_3 166       If you have a lime or hard water buildup clog...     135

In [7]:

embeddings_model_path = 'msmarco-distilbert-base-v4'
embeddings_model = SentenceTransformer(embeddings_model_path)
print(f'Default sequence length:{embeddings_model.max_seq_length}')


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Default sequence length:512


In [8]:
# This could take a bit of time
document_embeddings = compute_doc_embeddings(df, embeddings_model)

# An example embedding:
example_entry = list(document_embeddings.items())[0]
# print(example_entry)


In [9]:
print(f'Total documents, {len(document_embeddings)}')

Total documents, 149


In [10]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    We use cosine similarity 
    """
    return util.cos_sim(x,y)

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array], model: SentenceTransformer) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_query_embedding(query, model)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities



In [11]:
# ask a question and get top_k relevant text
top_k = 5
by_semantic_relevance = order_document_sections_by_query_similarity("Why is my dishwasher leaking?", document_embeddings, embeddings_model)[:top_k]
for i in range(top_k):
    index = by_semantic_relevance[i][1]  # index
    print(f"index: {index} {[i]}:  {df.loc[index]['content']}")


index: ('Chapter_4', 190) [0]:   There are two general areas where you will commonly find water leaking from a dishwasher. The first is from around the door; water will generally show up on the kitchen floor in front of the dishwasher. The second is from some component under the tub. The most common sources are pump seals and water valves, though it really can come from just about anything else under there, such as heater mounts, float switches, dryer fans, hoses, etc.
index: ('Chapter_2', 81) [1]:   POOR WASH QUALITY: This is the most common complaint in a dishwasher. It covers a lot of different specific symptoms, from spotting, film or etching of the dishes to food left on dishes. It is discussed in detail in Chapter 3. NO POWER; MOTOR WON'T START: First check the house breaker or fuse. Next, make sure that the dishwasher is plugged into the correct wall socket. (See the note in section 5-2 for an explanation)
index: ('Chapter_4', 191) [2]:   A slow, under-tub leak may go years with

In [12]:
# ask another question
top_k = 5
by_semantic_relevance = order_document_sections_by_query_similarity("Why is my dish washer not cleaning well?", document_embeddings, embeddings_model)[:top_k]
for i in range(top_k):
    index = by_semantic_relevance[i][1]  # [0] similarity [1]index
    print(f"{[i]} {df.loc[index]['content']}")

[0]  NOTE: This Chapter assumes that the motor is running and you hear water whooshing around inside the machine! If not, go back to chapter 2 and troubleshoot ! Poor wash quality mainly involves heavy spotting and filming or etching of the dishes. It can usually be traced to cold water or excessively hot, hard or soft water, poor detergent, or inadequate drying. Washing dishes in a dishwasher is not just a matter of blowing hot water at them. It is not just simply a mechanical or hydraulic process. It is also a chemical process. The chemicals you use in your dishwasher, from detergent to rinse agent, are extremely critical. And the temperature of the water that dissolves and carries these chemicals is critical as well.
[1]  POOR WASH QUALITY: This is the most common complaint in a dishwasher. It covers a lot of different specific symptoms, from spotting, film or etching of the dishes to food left on dishes. It is discussed in detail in Chapter 3. NO POWER; MOTOR WON'T START: First che

In [13]:
# TODO what is this?
from transformers import GPT2TokenizerFast
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
separator_len = len(tokenizer.tokenize(SEPARATOR))

f"Context separator contains {separator_len} tokens"

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

'Context separator contains 3 tokens'

In [14]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame, embeddings_model: SentenceTransformer) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings, embeddings_model)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]

        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            # print(f'Enough context---run out of length of {MAX_SECTION_LEN}')
            break
            
        chosen_sections.append(SEPARATOR + document_section['content'].replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    # print(f"Selected {len(chosen_sections)} document sections:")
    # print("\n".join(chosen_sections_indexes))
    
    # The context
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [13]:
# Try out the prompt
prompt = construct_prompt(
    "Why is my dish wasker leaking?",
    document_embeddings,
    df,
    embeddings_model
)

print("===\n", prompt)

===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  There are two general areas where you will commonly find water leaking from a dishwasher. The first is from around the door; water will generally show up on the kitchen floor in front of the dishwasher. The second is from some component under the tub. The most common sources are pump seals and water valves, though it really can come from just about anything else under there, such as heater mounts, float switches, dryer fans, hoses, etc.
*  POOR WASH QUALITY: This is the most common complaint in a dishwasher. It covers a lot of different specific symptoms, from spotting, film or etching of the dishes to food left on dishes. It is discussed in detail in Chapter 3. NO POWER; MOTOR WON'T START: First check the house breaker or fuse. Next, make sure that the dishwasher is plugged into the correct wall socket. (See the note

In [15]:

COMPLETIONS_MODEL = "gpt-4o"
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 200,
    "model": COMPLETIONS_MODEL,
}

### Answering the question from a context

In [25]:
import os
import getpass
from openai import OpenAI
OPENAI_API_KEY = getpass.getpass("Enter OpenAI API key")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# print(os.environ.get("OPENAI_API_KEY"))
client = OpenAI()

Enter OpenAI API key ········


In [28]:

def answer_query_from_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df,
        embeddings_model
    )
    
    if show_prompt:
        print(prompt)

    response =client.chat.completions.create(
                model=COMPLETIONS_MODEL,
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")



In [29]:
# response = answer_query_with_context("Who won the 2020 Summer Olympics men's high jump?", df, document_embeddings)
from scipy.__config__ import show


response = answer_query_from_context("What are the ways to prevent water leakage?", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n" {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  A third source of water "leakage" (though it really isn't a leak) is from the air gap. The air gap is an anti-backflow device installed in the drain line, to prevent the dishwasher from accidentally siphoning fluid from your house's sewer line back into the dishwasher tub. If this air gap or the house's drain line becomes clogged, water can run out of the air vent, and generally it runs straight into your sink. Note, however, that this will only occur when the dishwasher is operating in the "pumpout" mode, trying to drain the tub. There is an illustration and discussion of the air gap in section 3-1.
*  AIR GAP: There is also an anti-siphon device, called an air gap, built into the drain line. It is required by law in most installations. It prevents accidental backflow (siphoning) into the dishwasher from the house drain l

TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given

In [18]:
response = answer_query_from_context("Explain the various cycles?", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  Sometimes you can eliminate possibilities just simply be studying the wiring diagram for a few minutes. For example, let's say your timer is not advancing during any cycle. Following the gray line in figure 6-C, the timer is fed through three different circuits. Sometimes it gets its power through timer switch 22, sometimes it gets power from switch "D" inside the pushbutton selector switch, and it can also get power from the thermostat. It is unlikely that all three switches are bad, so the likelihood is that the timer motor itself is on the fritz.
*  You should see continuity make and break at least once in the cycle; usually several times. If it doesn't, the internal contacts are bad; replace the timer. Repeat the process with the test leads between the W-V and the O-BK terminals. You can drive yourself crazy trying to 

In [19]:

response = answer_query_from_context("Why is my dishwasher leaking?", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n" {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  There are two general areas where you will commonly find water leaking from a dishwasher. The first is from around the door; water will generally show up on the kitchen floor in front of the dishwasher. The second is from some component under the tub. The most common sources are pump seals and water valves, though it really can come from just about anything else under there, such as heater mounts, float switches, dryer fans, hoses, etc.
*  POOR WASH QUALITY: This is the most common complaint in a dishwasher. It covers a lot of different specific symptoms, from spotting, film or etching of the dishes to food left on dishes. It is discussed in detail in Chapter 3. NO POWER; MOTOR WON'T START: First check the house breaker or fuse. Next, make sure that the dishwasher is plugged into the correct wall socket. (See the note in s

In [20]:
response = answer_query_from_context("What's the biggest thing to worry about?", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n" {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  Many household handymen will tackle just about any job, but are downright scared of electricity. It's true that you have to be extra careful around live circuits due to the danger of getting shocked. But there's no mystery or voodoo about what we'll be doing. I'll cut out most of the electric theory for you. If you're one of those folks who's a bit timid around electricity, all I can say is read on, and don't be too nervous. It will come to you.
*  The solution is to replace the defective motor or switch. If it's the timer, you may be able to get a rebuilt one to save a few bucks. WATER LEAKING ONTO FLOOR: If water is coming out the front of the machine, it's usually leaky door seals, but there are a few other suspects. The wrong soap can cause suds, which can leak out even if the seals are good. There are also some design

In [23]:
response = answer_query_from_context("What are the sources of water leaks?", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n" {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  There are two general areas where you will commonly find water leaking from a dishwasher. The first is from around the door; water will generally show up on the kitchen floor in front of the dishwasher. The second is from some component under the tub. The most common sources are pump seals and water valves, though it really can come from just about anything else under there, such as heater mounts, float switches, dryer fans, hoses, etc.
*  A third source of water "leakage" (though it really isn't a leak) is from the air gap. The air gap is an anti-backflow device installed in the drain line, to prevent the dishwasher from accidentally siphoning fluid from your house's sewer line back into the dishwasher tub. If this air gap or the house's drain line becomes clogged, water can run out of the air vent, and generally it runs 

In [22]:
response = answer_query_from_context("Tell me a bit about the warranty", df, document_embeddings, show_prompt=True)

answer = response[0:len(response) + 1].split('A:')[-1].strip()

print(f'====Answer\n" {answer}')

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

*  If they genuinely try to help you fix it yourself, and you find that you're unable to, they may be the best place to look for service. Here's a hot tip: after what I just said, if they sold you this book, then I'll just about guarantee they're genuinely interested in helping do-it-yourselfers. When you go into the store, have ready the make, model and serial number from the nameplate of the dishwasher.
*  Find yourself a good appliance parts dealer. You can find them in the yellow pages under the following headings: â¢ Appliances, Household, Major â¢ Appliances, Parts and Supplies â¢ Dishwashers, Domestic â¢ Appliances, Household, Repair and Service Call a few of them and ask if they are a dishwasher repair service, or if they sell parts, or both. Ask them if they offer free advice with the parts they sell (Occasionally, s